TO DO:
- Calculate network's metrics;
- Create node embeddings;
- Check if the graph is undirected;
- Use t-SNE and then visualize the representations alongside its respective classes;
- Separate the unlabeled classes in the Test set;

Questions:
- What is the difference between media_authro and commenter?
- Should I consider the graph as undirected?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
import missingno as msno
import ast

In [ ]:
medias = pd.read_csv("../data/medias.csv")

In [ ]:
medias.tail(3)

In [ ]:
print("The number of null values in each column\n%s" % medias.isna().sum())

In [ ]:
profiles = pd.read_csv("../data/profiles.csv")

In [ ]:
profiles.tail(3)

In [ ]:
msno.matrix(profiles)

In [ ]:
print("The number of anotated profiles:", profiles.profile_category.count())

There are 6 skewed classes. How to deal with it? Oversampling?

In [ ]:
profiles.profile_category.value_counts()

In [ ]:
usernames = profiles.profile_username.dropna()

In [ ]:
comments = pd.read_csv("../data/comments.csv").sample(10000)

In [ ]:
comments.tail(3)

create the neighbourhood for each username

In [ ]:
interactions = comments[['media_author', 'comment_mentioned_usernames']].values
vertices = set()
for user, neighbors in interactions:
    for neighbor in ast.literal_eval(neighbors):
        if user > neighbor:
            user, neighbor = neighbor, user
        vertices.add((user, neighbor))

In [ ]:
print("The number of null values in each column\n%s" % comments.isna().sum())

In [ ]:
graph = nx.Graph()
graph.add_nodes_from(usernames)
graph.add_edges_from(vertices)

nx.draw(graph, node_size=10)
plt.title("Interaction Between users (sampled)")
plt.show()